## Setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# get modules
from executive_summary import *
from cumulative_accounting import *
from air_quality import *
from fisheries import *
from noise import *
from recreation import *
from scenic_resources import *
from soil_conservation import *
from transportation_sustainablecommunities import *
from vegetation_preservation import *
from water_quality import *
from wildlife import *
localpath = pathlib.Path.cwd()

## Executive Summary

In [ ]:
# tables and charts for the executive summary page


## Cumulative Accounting

In [ ]:
# tables and charts for the cumulative accounting web page

In [ ]:
# summarize existing development by land capability category
summarize_landcap_by_parcel('2019')
df19, sdf19 = get_summary('2019')

In [11]:
# summarize existing development by land capability category
summarize_landcap_by_parcel('2023')
df23, sdf23 = get_summary('2023')

c:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\Lib\site-packages\pandas\core\dtypes\cast.py:1066: RuntimeWarning: invalid value encountered in cast
  if (arr.astype(int) == arr).all():
c:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\Lib\site-packages\pandas\core\dtypes\cast.py:1091: RuntimeWarning: invalid value encountered in cast
  if (arr.astype(int) == arr).all():


In [ ]:
df = get_old_dev_cap()
df = pd.melt(df, id_vars=['MajorityLandCapability_NRCS'], value_vars=['RES_2019', 'TAU_2019', 'CFA_2019'], var_name='DevelopmentType', value_name='Value')
df = df.groupby(['MajorityLandCapability_NRCS', 'DevelopmentType'], dropna=False).sum().reset_index()
# set Value to int
df['Value'] = df['Value'].astype(int)
dfOld19 = df

,MajorityLandCapability_NRCS,DevelopmentType,Value
0,,CFA_2019,11968
1,,RES_2019,36
2,,TAU_2019,32
3,1A,CFA_2019,57387
4,1A,RES_2019,387
5,1A,TAU_2019,0
6,1B,CFA_2019,1430215
7,1B,RES_2019,5521
8,1B,TAU_2019,1750
9,1C,CFA_2019,595050


In [ ]:
df19_landcap.

AttributeError: 'DataFrame' object has no attribute 'TouristAccommodation_Units'

In [13]:
# melt by category and sum Residential_Units, TouristAccommodation_Units, and CommercialFloorArea_SqFt
df = pd.melt(sdf19, id_vars=['Majority_Land_Capab'], value_vars=['Residential_Units', 'TouristAccommodation_Units', 'CommercialFloorArea_SqFt'], var_name='DevelopmentType', value_name='Value')
df = df.groupby(['Majority_Land_Capab', 'DevelopmentType'], dropna=False).sum().reset_index()
df19_landcap = df

In [14]:
# melt by category and sum Residential_Units, TouristAccommodation_Units, and CommercialFloorArea_SqFt
df = pd.melt(sdf23, id_vars=['Majority_Land_Capab'], value_vars=['Residential_Units', 'TouristAccommodation_Units', 'CommercialFloorArea_SqFt'], var_name='DevelopmentType', value_name='Value')
df = df.groupby(['Majority_Land_Capab', 'DevelopmentType'], dropna=False).sum().reset_index()
df23_landcap = df

## Air Quality

In [ ]:
# get PM 2.5
df = get_airquality_data_sql()
# plot PM 2.5
plot_pm2_5_annual(df, draft=False)

In [ ]:
# get PM 2.5
df = get_airquality_data_sql()
# plot PM 2.5
plot_pm2_5_24hour(df, draft=False)

In [ ]:
# get PM 10
df = get_airquality_data_sql()
#Get PM 10 24 hour high
plot_pm10_24hr(df, draft= False)

In [ ]:
# get PM 10
df = get_airquality_data_sql()
#Plot PM 10 Average Annual Concentrations
plot_pm10_annual(df, draft= False)

In [ ]:
# get 03
df = get_airquality_data_sql()
#Plot 03 1 hour high
plot_o3_1hour_high(df, draft= False)

In [ ]:
# get CO
df = get_airquality_data_sql()
#Plot CO 1 hour high
plot_co_8hour_avg(df, draft= True)

In [ ]:
#Get 50th regionalVisibility Bliss 
df = get_airquality_data_sql()
#Plot Visibility at Bliss
plot_50_Bliss_vis(df, draft= False)

In [ ]:
#Get 90th regionalVisibility Bliss 
df = get_airquality_data_sql()
#Plot 90th percentile Visibility at Bliss
plot_90_Bliss_vis(df, draft= False)

In [ ]:
#Get 50th Visibility subregional slt
df = get_airquality_data_sql()
#Plot 50th Visibility subregional slt
plot_50_SLT_vis(df, draft= False)

In [ ]:
#Get 90th Visibility subregional slt
df = get_airquality_data_sql()
#Plot 90th Visibility subregional slt
plot_90_SLT_vis(df, draft= False)

In [ ]:
#Get NOx data
df = get_NOX_data_sql()
#Plot NOX
plot_NOx(df, draft= False)

In [ ]:
#Get winter traffic data
df = get_wintertraffic_data_sql()
plot_winter_traffic(df, draft= False)

In [ ]:
# mid lake dissolved nitrogen data and chart
df = get_midlake_dissolved_nitrogen_data()
plot_midlake_dissolved_nitrogen(df, draft=False)

## Fisheries

In [ ]:
df = get_fishhab()
plot_fishhab(df, draft=False)

## Noise

In [ ]:
# plot commercial noise
# get data
df = get_plannoise_data_sql()
# plot data
plot_commercial(df, draft=False)

In [ ]:
# plot hotel/motel noise data
# get data
df = get_plannoise_data_sql()
# plot data
plot_hotelmotel(df, draft=True)

In [ ]:
# high density residential noise data
# get data
df = get_plannoise_data_sql()
# plot data
plot_highdensityresidential(df, draft=False)

In [ ]:
# plot industrial noise data
# get data
df = get_plannoise_data_sql()
# plot data
plot_industrial(df, draft=False)

In [ ]:
# plot low density residential noise data
# get data
df = get_plannoise_data_sql()
# plot data
plot_lowdensity(df, draft=False)

In [ ]:
# plot rural noise data
# get data
df = get_plannoise_data_sql()
# plot data
plot_rural(df, draft=False)

In [ ]:
# plot urban outdoor recreation noise
# get data
df = get_plannoise_data_sql()
# plot data
plot_rec(df, draft=False)

In [ ]:
# plot wilderness and roadless area noise
# get data
df = get_plannoise_data_sql()
# plot data
plot_wilderness(df, draft=False)

In [ ]:
# plot watercraft shore noise
# get data
df = get_shorenoise_data_sql()
# plot data
plot_watercraft(df, draft=False)

## Scenic Resources

In [ ]:
# plot scenic corridor roadway attainment
# get data
df = get_scenic_corridor_rating()
# plot data
plot_scenic_corridor_attainment_roadway(df, draft=False)

In [ ]:
# plot scenic corridor shoreline attainment
# get data
df = get_scenic_corridor_rating()
# plot data
plot_scenic_corridor_attainment_shoreline(df, draft=False)

In [ ]:
# plot scenic viewpoint average rating
# get data
df = get_scenic_viewpoint_rating()
# plot data
plot_scenic_viewpoint_roadway_rating(df, draft=True)

In [ ]:
# plot scenic viewpoint shoreline average rating
# get data
df = get_scenic_viewpoint_rating()
# plot data
plot_scenic_viewpoint_shoreline_rating(df, draft=True)

## Soil Conservation

In [ ]:
dfImpChg, dfImp2019 = get_soil_conservation_data_sql()
dfImpOld = transform_soil_conservation_data(dfImpChg, dfImp2019)
dfImpNew = transform_new_coverage_data()
df = add_new_coverage(dfImpOld, dfImpNew)

In [ ]:
# plot 1A - soil conservation
plot_soil_conservation(df, landcap = '1A', draft=False)
plot_soil_conservation(df, landcap = '1B', draft=False)
plot_soil_conservation(df, landcap = '1C', draft=False)
plot_soil_conservation(df, landcap = '2', draft=False)
plot_soil_conservation(df, landcap = '3', draft=False)
plot_soil_conservation(df, landcap = '4', draft=False)
plot_soil_conservation(df, landcap = '5', draft=False)
plot_soil_conservation(df, landcap = '6', draft=False)
plot_soil_conservation(df, landcap = '7', draft=False)

In [ ]:
# Get SEZ Data
df = get_SEZ_data_web()
# Make SEZ/Wetland Restoration Chart
plot_SEZ_scores(df,draft=False) 

## Vegetation

In [ ]:
# Vegetation Preservation Abundance
# get veg data
df =get_ecobject_caldor_identity_data()
# plot veg data
plot_veg_abundance(df, draft=True)

In [ ]:
# Vegetation Seral Stage
# get veg data
df =get_ecobject_caldor_identity_data()
# plot veg data
plot_seral_stage(df, draft=False)

In [ ]:
# Vegetatoin Composition
# get veg data
df =get_ecobject_caldor_identity_data()
# plot veg data
plot_veg_composition(df, draft=True)

In [ ]:
#Get Sensitive plant data
#TDraba, CDraba, Lewisia = get_sensitiveplant_xlsx()
df= get_TDraba_xslx()
#plot_TDraba
plot_TDraba(df, draft=True)

In [ ]:
#Get Cup Draba subpopulation data
df= get_CDraba_xslx()
#plot CDraba
plot_CDraba(df, draft=True)

In [ ]:
#Get Long Petaled Lewisia data
df= get_lewisia_xslx()
#plot CDraba
plot_lewisia(df, draft=True)

In [ ]:
# Relative Abundance of Red Fir Forest In Seral Stages Other Than Mature
# get data
df = get_ecobject_caldor_identity_data()
# plot red fir forest chart
plot_redfir(df, draft=False)

In [ ]:
# Relative Abundance of Yellow Pine Forest In Seral Stages Other Than Mature
# get data
df = get_ecobject_caldor_identity_data()
# plot red fir forest chart
plot_yellowpine(df, draft=False)

In [ ]:
# Relative Abundance of Meadows And Wetland Vegetation Types
# get data
df = get_ecobject_caldor_identity_data()
# plot
plot_wetland(df, draft=False)

In [ ]:
# Relative Abundance of Shrub Vegetation Type
# get data
df = get_ecobject_caldor_identity_data()
# plot
plot_shrub(df, draft=False)

In [ ]:
# Relative Abundance of Deciduous Riparian Vegetation
# get data
df = get_ecobject_caldor_identity_data()
# plot
plot_deciduous(df, draft=True)

In [ ]:
# Pattern: Stand Composition And Age
# get data
df = get_ecobject_caldor_identity_data()
# plot red fir forest chart
plot_veg_composition(df, draft=True)

In [ ]:
# Consistency with Bailey Land Capability System
# get data
dfImpChg, dfImp2019 = get_soil_conservation_data_sql()
# plot land capability chart
plot_landcapability(dfImpChg, draft=False)

In [ ]:
# Appropriate Management Practices
df = get_data_forest_fuel()
plot_forest_fuel(df, draft=True)

In [ ]:
# Total Old Growth
df = get_old_growth()
plot_old_growth(df, draft=False)

In [ ]:
# Old Growth in Sub-Alpine Zone
df = get_ecobject_caldor_identity_data()
plot_seral_subalpine(df, draft=False)

In [ ]:
# Old Growth in Upper Montane Zone
df = get_ecobject_caldor_identity_data()
plot_seral_upper_montane(df, draft=False)

In [ ]:
# Old Growth in Montane Zone
df = get_ecobject_caldor_identity_data()
plot_seral_montane(df, draft=False)

In [ ]:
#Tahoe Yellow Cress Chart
df = get_TYC_data_sql()
plot_TYC(df, draft=False)

## Water Quality

In [ ]:
# secchi depth data and chart
df = get_secchi_data_sql()
# df = get_secchi_data_web()
plot_secchi_depth(df, draft=False)

In [ ]:
# watercraft inspections
df = get_watercraft_inspection_data_web()
plot_watercraft_inspections(df, draft=False)

In [ ]:
# get water quality data
df = get_waterquality_data()
# plot total nitrogetn
plot_total_nitrogen(df, draft=False)
plot_total_phosphorus(df, draft=False)
plot_suspended_sediment(df, draft=False)

In [ ]:
# get total load reduction data
df = get_phosphorus_load_reduction()
plot_phosphorus_load_reduction(df, draft=False)

In [ ]:
# get total load reduction data
df = get_nitrogen_load_reduction()
plot_nitrogen_load_reduction(df, draft=False)

In [ ]:
# get total load reduction data
df = get_sediment_load_reduction()
plot_sediment_load_reduction(df, draft=False)

In [ ]:
# periyphyton data and chart
df = get_periphyton_data()
plot_periphyton(df, draft=False)

In [ ]:
# nearshore turbidity data and chart
df = get_nearshore_turbidity_data()
plot_nearshore_turbidity(df, draft=True)

In [ ]:
df = get_ais_infestation_data_sql()
plot_ais_infestation(df, draft=True)

## Wildlife

In [ ]:
# goshawk data and chart
df = get_wildlife_data_web()
plot_goshawk_data(df, draft=False)

In [ ]:
# osprey data and chart
df = get_wildlife_data_web()
plot_osprey_data(df, draft=True)

In [ ]:
# bald eagle winter roost data and chart
df = get_wildlife_data_web()
plot_bald_eagle_winter(df, draft=False)

In [ ]:
df = get_bald_eagle_data_wt_estimate_sql()
plot_bald_eagle_winter_est(df, draft=False)

In [ ]:
df = get_peregrine_falcon_data_wt_estimate_sql()
plot_pfalcon_data_wt_estimate(df, draft=False)

In [ ]:
df = get_osprey_data_wt_estimate_sql()
plot_osprey_data_wt_estimate(df, draft=False)

In [ ]:
# peregine falcon data and chart
df = get_wildlife_data_web()
plot_falcon_data(df, draft=False)

In [ ]:
# bald eagle summer roost data and chart
df = get_wildlife_data_web()
plot_bald_eagle_summer(df, draft=False)

In [ ]:
#waterfowl data and chart
df = get_waterfowl_data_web()
plot_waterfowl_data(df, draft=False)